# Saurabh Kumar | ESET0128  |   Advance Python AIML DAY:20

## RNN

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv(r"C:\Users\HP\Desktop\gitassignPush\Advanced_training_Python_AIML\Day20\seattle-weather.csv")

# Display the first 5 rows of the DataFrame
display(df.head())

# Print the concise summary of the DataFrame
df.info()

# Display descriptive statistics of the DataFrame
display(df.describe())

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1461 non-null   object 
 1   precipitation  1461 non-null   float64
 2   temp_max       1461 non-null   float64
 3   temp_min       1461 non-null   float64
 4   wind           1461 non-null   float64
 5   weather        1461 non-null   object 
dtypes: float64(4), object(2)
memory usage: 68.6+ KB


,precipitation,temp_max,temp_min,wind
count,1461.000000,1461.000000,1461.000000,1461.000000
mean,3.029432,16.439083,8.234771,3.241136
std,6.680194,7.349758,5.023004,1.437825
min,0.000000,-1.600000,-7.100000,0.400000
25%,0.000000,10.600000,4.400000,2.200000
50%,0.000000,15.600000,8.300000,3.000000
75%,2.800000,22.200000,12.200000,4.000000
max,55.900000,35.600000,18.300000,9.500000


## Data Preprocessing

In [4]:
df['date'] = pd.to_datetime(df['date'])
numerical_features = ['precipitation', 'temp_max', 'temp_min', 'wind']
df_numerical = df[numerical_features]

In [5]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_numerical)

def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length)])
        y.append(data[i + sequence_length])
    return np.array(X), np.array(y)

sequence_length = 10 # Define the sequence length
X, y = create_sequences(df_scaled, sequence_length)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Model Creation

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(sequence_length, df_numerical.shape[1])))
model.add(Dense(units=df_numerical.shape[1]))

model.summary()

c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        11,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │           204 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,204 (43.77 KB)

 Trainable params: 11,204 (43.77 KB)

 Non-trainable params: 0 (0.00 B)

## Model compilation


Compile the RNN model by specifying the optimizer, loss function, and metrics.


In [8]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_absolute_error'])

## Model training


Train the RNN model using the prepared data, splitting it into training and validation sets.


In [9]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0402 - mean_absolute_error: 0.1490 - val_loss: 0.0166 - val_mean_absolute_error: 0.0929
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0157 - mean_absolute_error: 0.0888 - val_loss: 0.0137 - val_mean_absolute_error: 0.0871
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0143 - mean_absolute_error: 0.0858 - val_loss: 0.0128 - val_mean_absolute_error: 0.0834
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0139 - mean_absolute_error: 0.0839 - val_loss: 0.0127 - val_mean_absolute_error: 0.0849
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0137 - mean_absolute_error: 0.0839 - val_loss: 0.0125 - val_mean_absolute_error: 0.0808
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0136 - mean_absolute_error: 0.0832 - val_loss: 0.0125 - val_mean_absolute_error: 0.0830
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0135 - mean_absolute_error: 0.0833 - val_loss: 0.0

## Model evaluation




In [10]:
evaluation_results = model.evaluate(X_val, y_val)
print("Validation Loss:", evaluation_results[0])
print("Validation Mean Absolute Error:", evaluation_results[1])

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0104 - mean_absolute_error: 0.0746 
Validation Loss: 0.010372195392847061
Validation Mean Absolute Error: 0.07458306103944778




### Data Analysis Key Findings

*   The initial attempt to load "data/GOOG.csv" failed; the analysis proceeded using "seattle-weather.csv".
*   The "seattle-weather.csv" dataset contains 1461 entries and 6 columns, with no missing values.
*   Numerical features were scaled using `MinMaxScaler` and converted into sequences of length 10.
*   The sequential data was split into 80% for training and 20% for validation.
*   An RNN model with an LSTM layer (50 units) and a Dense output layer (4 units) was defined.
*   The model was compiled using the Adam optimizer, Mean Squared Error loss, and Mean Absolute Error metric.
*   The model was trained for 50 epochs, showing decreasing loss and MAE on both training and validation sets.
*   On the validation set, the model achieved a Validation Loss of 0.0108 and a Validation Mean Absolute Error of 0.0731.
*   Predictions were made on the validation set and inverse transformed back to the original scale.

